# AIS Data Cleaning  
This notebook will serve as the primary dev environment for loading, cleaning, and preparing the AIS data from Janurary 1st 2025 in the waters around Denmark in the North Sea. The data is stored in one large CSV that contains all ships and is approximately 2.5 GBs for a single day. Breakdown of the cleaning process:  
1. Remove features that are unnecessary such as ROT, Heading, IMO, etc.
2. Impute any missing values for rows that are present in other rows for the same ship.
3. Drop features that contain large amounts of missing data.
4. Check whether the data appears to be single routes or not. Decide on a criteria for separating ship routes if more than one appears present.
5. Sort data into single ship routes. 

## TODO:  
Load in the other datasets. Stitch them together. Should they be put together before or after cleaning? After is probably more feasible without all the bad rows.

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import random
import datetime

In [22]:
# Function to draw tracks 
import descartes
import geopandas as gpd
import fiona
from shapely.geometry import Point, LineString, Polygon

%matplotlib inline

def draw_tracks(data, mmsi_list):
    data = data[data.MMSI.isin(mmsi_list)]
    points = [Point(x, y) for x, y in zip(data.LON, data.LAT)]
    points_df = gpd.GeoDataFrame(data, geometry=points, crs ="EPSG:4326")

    # treat each `ID` group of points as a line
    lines = points_df.groupby(['MMSI'])['geometry'].apply(lambda x:  LineString(x.tolist()))
    
    # store as a GeodataFrame and add 'ID' as a column (currently stored as the 'index')
    lines = gpd.GeoDataFrame(lines, geometry='geometry', crs="EPSG:4326") 
    lines.reset_index(inplace=True)
    lines.plot(column='MMSI', categorical=True, legend=True)

def draw_all_tracks(data):
    all_ships = list(data.columns.values)
    draw_tracks(data, all_ships)

def prepare_data_for_vis(data):
    uniques = np.unique(data.MMSI, return_counts=True)
    index_to_remove = list(np.where(uniques[1] == 1)[0])
    to_remove = uniques[0][index_to_remove]
    return data[data.MMSI.isin(to_remove) == False]

### Loading the Data

In [23]:
full_data = pd.read_csv("./data/aisdk-2025-01-01.csv")

In [24]:
full_data.head()

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,01/01/2025 00:00:00,Class A,219005904,56.041798,12.613100,Unknown value,NaN,0.0,279.7,NaN,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
1,01/01/2025 00:00:00,Base Station,2190064,56.716560,11.519037,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
2,01/01/2025 00:00:00,Class A,311000766,57.775863,7.901422,Under way using engine,-8.7,4.1,259.2,255.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
3,01/01/2025 00:00:00,Class A,265514800,55.613652,12.997382,Under way using engine,NaN,0.0,74.0,NaN,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
4,01/01/2025 00:00:00,AtoN,992111851,54.441580,7.678878,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN


In [25]:
full_data.shape

(14166456, 26)

In [26]:
full_data.columns

Index(['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude',
       'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO',
       'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length',
       'Type of position fixing device', 'Draught', 'Destination', 'ETA',
       'Data source type', 'A', 'B', 'C', 'D'],
      dtype='object')

In [27]:
full_data.isnull().sum()

# Timestamp                              0
Type of mobile                           0
MMSI                                     0
Latitude                                 0
Longitude                                0
Navigational status                      0
ROT                                4406351
SOG                                1198351
COG                                2127800
Heading                            3090271
IMO                                      0
Callsign                                 0
Name                               1071632
Ship type                                0
Cargo type                        11926152
Width                              1427359
Length                             1427769
Type of position fixing device           0
Draught                            3934997
Destination                           3723
ETA                                5617596
Data source type                         0
A                                  1471918
B          

In [28]:
full_data.IMO.unique()

array(['Unknown', '8918966', '1167312', ..., '8800157', '7931997',
       '9122007'], shape=(1045,), dtype=object)

In [59]:
ais.head()

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
2,01/01/2025 00:00:00,Class A,311000766,57.775863,7.901422,Under way using engine,-8.7,4.1,259.2,255.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
12,01/01/2025 00:00:00,Class A,304010297,54.603442,11.169567,Under way using engine,0.0,4.3,292.6,281.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
13,01/01/2025 00:00:00,Class A,304010297,54.603442,11.169567,Under way using engine,0.0,4.3,292.6,281.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
14,01/01/2025 00:00:00,Class A,304010297,54.603442,11.169567,Under way using engine,0.0,4.3,292.6,281.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
15,01/01/2025 00:00:00,Class A,314583000,57.874588,10.198435,Under way using engine,NaN,2.5,278.1,276.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN


### Step 1: Remove duplicate MMSIs

In [29]:
ais = full_data.dropna(subset=['MMSI','IMO']) # Drop rows with missing IMO and MMSI because no way to identify ship

In [30]:
ais.shape

(14166456, 26)

In [31]:
mmsi_col = "MMSI"
imo_col  = "IMO"

# treat '' or 'UNKNOWN' (case-insensitive) as NA
ais[imo_col] = ais[imo_col].replace(
    {"": None, "UNKNOWN": None, "Unknown": None, "unknown": None}
)

# map each MMSI to the set of *known* IMOs it appears with
mmsi_to_imo = (
    ais.dropna(subset=[imo_col])
       .groupby(mmsi_col)[imo_col]
       .agg(lambda s: s.unique().tolist())
)

def resolve_imo(imolist):
    if len(imolist) == 1:
        return imolist[0]        # unique ⇒ OK
    else:
        print("Problem MMSI")
        return None              # 0 or >1 ⇒ unresolved

mmsi_to_single_imo = mmsi_to_imo.apply(resolve_imo)

In [32]:
ais.IMO.isna().sum()

np.int64(6180176)

In [33]:
# create a Series we can map with
impute_map = mmsi_to_single_imo.dropna()  # keep only MMSI with a unique IMO

# fill NA IMO values where possible
mask_missing = ais[imo_col].isna()
ais.loc[mask_missing, imo_col] = (
    ais.loc[mask_missing, mmsi_col]
       .map(impute_map)          # map returns NaN if MMSI not in `impute_map`
)

In [34]:
ais.IMO.isna().sum()

np.int64(6147440)

In [35]:
# Clean ais without ships that did not report an IMO
ais = ais[ais[imo_col].notna()] 

In [36]:
ais.shape

(8019016, 26)

In [37]:
# Save to individual CSVs
for imo, df_vessel in ais.groupby('IMO'):
    if df_vessel.shape[0] > 100000:
        print("Size of trajectory: {}".format(df_vessel.shape[0]))
    df_vessel.to_csv(f"./data/trajectories/IMO_{imo}.csv", index=False)

### Step 2: Remove outliers in each trajectory

In [38]:
import contextily as ctx

In [39]:
from pyproj import Transformer

# Geodetic WGS84  →  metric UTM 32N
to_utm = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True).transform

In [40]:
LAT_MIN, LAT_MAX = -85.05113, 85.05113     # Web-Mercator safe limits
LON_MIN, LON_MAX = -180.0, 180.0

def clean_latlon(df, lat_col='Latitude', lon_col='Longitude'):
    mask_lat = df[lat_col].between(LAT_MIN, LAT_MAX)
    mask_lon = df[lon_col].between(LON_MIN, LON_MAX)
    cleaned   = df[mask_lat & mask_lon].copy()
    dropped_n = len(df) - len(cleaned)
    if dropped_n:
        print(f"Dropped {dropped_n} rows with impossible lat/lon.")
    return cleaned

In [41]:
import glob, os
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from pyproj import Transformer

# ------------------------------------------------------------
LAT_COL, LON_COL = "Latitude", "Longitude"
EPS_METERS       = 300          # DBSCAN ε
MIN_SAMPLES      = 3            # DBSCAN minPts
MIN_VALID_PTS    = MIN_SAMPLES+1  # skip smaller tracks

in_dir  = Path("./data/trajectories")
out_dir = Path("./data/trajectories_clean")
out_dir.mkdir(parents=True, exist_ok=True)

proj = Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)


# ------------------------------------------------------------
files = sorted(glob.glob(str(in_dir / "IMO_*.csv")))

for fp in tqdm(files, desc="cleaning trajectories"):
    df = pd.read_csv(fp)
    #print("Size of file: {}".format(df.shape[0]))
    df = clean_latlon(df).dropna(subset=[LAT_COL, LON_COL])

    # if after cleaning we have almost no points → skip / drop
    if len(df) < MIN_VALID_PTS:
        # choose one: either skip completely or save it unchanged
        # 1) skip:
        print("Trajectory too small")
        continue
        # 2) keep a copy (unfiltered) so downstream code sees it:
        #out_fp = out_dir / Path(fp).name
        #df.to_csv(out_fp, index=False)
        #continue

    # project to metres
    x, y   = proj.transform(df[LON_COL].values, df[LAT_COL].values)
    coords = np.column_stack([x, y])

    # DBSCAN on tracks with enough points
    labels = DBSCAN(
        eps=EPS_METERS,
        min_samples=MIN_SAMPLES,
        metric="euclidean"
    ).fit_predict(coords)

    df["label"] = labels
    df_clean    = df[df["label"] != -1].drop(columns="label")

    # optional: if *all* points became noise, you might drop the file
    if df_clean.empty:
        continue

    df_clean.to_csv(out_dir / Path(fp).name, index=False)

cleaning trajectories:   0%|▏                                                                | 3/1044 [00:00<02:28,  6.99it/s]

Dropped 1 rows with impossible lat/lon.
Dropped 1 rows with impossible lat/lon.


cleaning trajectories:   1%|▌                                                                | 9/1044 [00:02<05:19,  3.24it/s]

Dropped 3 rows with impossible lat/lon.
Dropped 1 rows with impossible lat/lon.


cleaning trajectories:   3%|█▉                                                              | 31/1044 [00:04<01:33, 10.89it/s]

...

Dropped 2 rows with impossible lat/lon.


cleaning trajectories:  97%|███████████████████████████████████████████████████████████▉  | 1010/1044 [03:22<00:05,  6.73it/s]

Dropped 3 rows with impossible lat/lon.


cleaning trajectories: 100%|██████████████████████████████████████████████████████████████| 1044/1044 [03:26<00:00,  5.06it/s]

Dropped 1 rows with impossible lat/lon.
Dropped 1 rows with impossible lat/lon.


### Step 3: Removing stationary trips

In [45]:
import glob, os, pathlib, math
from haversine import haversine

In [48]:
# -----------------------------------------------------------
# 1. parameters – ADJUST as you wish
# -----------------------------------------------------------
SRC_DIR   = "./data/trajectories_clean"       # input          (CSV files)
DST_DIR   = "./data/moving_trajectories"     # output (only “moving” ones)

MIN_DURATION_MIN  = 30       # track must span ≥ 30 min
MIN_DISPLACEMENT_KM = 5.0    # start–end distance  ≥ 5 km
MIN_TRACK_LEN_KM   = 5.0     # sum of segment lengths ≥ 5 km
LAT_COL, LON_COL   = "Latitude", "Longitude"
TIME_COL           = "# Timestamp"           # adjust to your column name

pathlib.Path(DST_DIR).mkdir(exist_ok=True, parents=True)

# -----------------------------------------------------------
# 2. helper to get simple movement statistics
# -----------------------------------------------------------
def track_stats(df):
    """
    Returns duration (minutes), displacement_km, track_len_km
    `df` must be sorted by timestamp already.
    """
    # duration
    t0, t1 = pd.to_datetime(df[TIME_COL].iloc[[0, -1]])
    duration_min = (t1 - t0).total_seconds() / 60.0

    # displacement (great-circle between first & last point)
    start = (df[LAT_COL].iloc[0], df[LON_COL].iloc[0])
    end   = (df[LAT_COL].iloc[-1], df[LON_COL].iloc[-1])
    displacement_km = haversine(start, end)

    # track length = sum of distance between consecutive points
    coords = df[[LAT_COL, LON_COL]].to_numpy()
    track_len_km = sum(haversine(tuple(coords[i-1]), tuple(coords[i]))
                       for i in range(1, len(coords)))

    return duration_min, displacement_km, track_len_km

# -----------------------------------------------------------
# 3. sweep through files
# -----------------------------------------------------------
src_files = sorted(glob.glob(os.path.join(SRC_DIR, "IMO_*.csv")))
print(len(src_files))
kept, skipped = 0, 0

for fp in tqdm(src_files, desc="Filtering stationary tracks"):
    df = pd.read_csv(fp)

    # sort by time for safety
    df = df.sort_values(TIME_COL).reset_index(drop=True)

    # --------- compute stats ----------
    dur_min, disp_km, track_km = track_stats(df)

    # --------- keep / discard ----------
    if (dur_min >= MIN_DURATION_MIN and
        disp_km  >= MIN_DISPLACEMENT_KM and
        track_km >= MIN_TRACK_LEN_KM):

        out_fp = os.path.join(DST_DIR, os.path.basename(fp))
        df.to_csv(out_fp, index=False)
        kept += 1
    else:
        skipped += 1

print(f"✔ Done.  Kept {kept} moving tracks, skipped {skipped} stationary ones.")

1038


Filtering stationary tracks: 100%|████████████████████████████████████████████████████████| 1038/1038 [00:43<00:00, 23.65it/s]

✔ Done.  Kept 447 moving tracks, skipped 591 stationary ones.


In [55]:
def plot_track(df,
               *,
               zoom=11,            # basemap zoom   (6-18, ↑ = closer)
               margin_m=2_000,     # extra metres around the track
               figsize=(8, 8),
               pts_kw=None,        # styling dict for points/line
               basemap=ctx.providers.OpenStreetMap.Mapnik,
               title=None):
    """
    Parameters
    ----------
    df : pandas.DataFrame
        Must contain 'Latitude' and 'Longitude' in WGS-84 degrees.
    zoom : int
        Slippy-tile zoom level for the basemap.
    margin_m : float
        Padding (metres) added to each side of the track’s bounding box.
    pts_kw : dict
        Extra kwargs passed to GeoPandas `.plot()` for the track
        (e.g. {'color':'crimson','markersize':12,'linewidth':1}).
    """
    if pts_kw is None:
        pts_kw = {"color": "crimson", "markersize": 18}

    # ---- 1. convert to GeoDataFrame in Web-Mercator (EPSG:3857) ----------
    gdf = gpd.GeoDataFrame(
        geometry=[Point(lon, lat) for lon, lat in zip(df.Longitude, df.Latitude)],
        crs="EPSG:4326"
    ).to_crs(epsg=3857)

    # ---- 2. figure & basemap --------------------------------------------
    fig, ax = plt.subplots(figsize=figsize)
    xmin, ymin, xmax, ymax = gdf.total_bounds
    ax.set_xlim(xmin - margin_m, xmax + margin_m)
    ax.set_ylim(ymin - margin_m, ymax + margin_m)

    ctx.add_basemap(ax, source=basemap, zoom=zoom, crs="EPSG:3857", zorder=1)

    # ---- 3. plot the track ----------------------------------------------
    gdf.plot(ax=ax, zorder=2, **pts_kw)

    ax.set_axis_off()
    if title:
        ax.set_title(title)
    fig.tight_layout()
    plt.show()

In [ ]:
# folder that contains your cleaned tracks
clean_dir = "./data/moving_trajectories"
csv_files = sorted(glob.glob(f"{clean_dir}/IMO_*.csv"))

# pick (say) three files to preview
for fp in random.sample(csv_files, k=3):
    df  = pd.read_csv(fp)
    imo = pathlib.Path(fp).stem      # 'IMO_1234567'
    plot_track(df, title=imo, zoom=10, margin_m=3_000)

In [60]:
df

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,01/01/2025 00:01:50,Class A,314525000,56.813753,10.631133,At anchor,0.0,0.0,0.0,199.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
1,01/01/2025 00:01:50,Class A,314525000,56.813753,10.631133,At anchor,0.0,0.0,0.0,199.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
2,01/01/2025 00:03:14,Class A,314525000,56.813753,10.631133,At anchor,0.0,0.0,0.0,199.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0
3,01/01/2025 00:04:49,Class A,314525000,56.813737,10.631172,At anchor,0.0,0.1,0.0,204.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0
4,01/01/2025 00:04:49,Class A,314525000,56.813737,10.631172,At anchor,0.0,0.1,0.0,204.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7407,01/01/2025 23:59:27,Class A,314525000,57.805335,10.935092,Under way using engine,0.0,7.6,298.6,295.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0
7408,01/01/2025 23:59:31,Class A,314525000,57.805402,10.934857,Under way using engine,0.0,7.6,303.1,296.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0
7409,01/01/2025 23:59:34,Class A,314525000,57.805453,10.934682,Under way using engine,0.0,7.7,299.0,295.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0
7410,01/01/2025 23:59:41,Class A,314525000,57.805577,10.934278,Under way using engine,0.0,7.6,294.8,295.0,...,108.0,GPS,7.2,ILHFA,16/01/2025 14:00:00,AIS,93.0,15.0,13.0,5.0


In [61]:
df.IMO

0       9218193
1       9218193
2       9218193
3       9218193
4       9218193
         ...   
7407    9218193
7408    9218193
7409    9218193
7410    9218193
7411    9218193
Name: IMO, Length: 7412, dtype: int64

## Step 4: Create distinct trips  
I have been able to separate all trips by day and into their own respective files. Now, I need to be able to stitch these files together for a single IMO and create the distinct tracks. 

In [72]:
from datetime import datetime, timedelta
import re

In [74]:
BASE_DIR = "../notebooks/data/trips"
imo_set = set()

# Regex to extract IMO number from filenames like "IMO_9431234.csv"
pattern = re.compile(r"IMO_(\d+)\.csv")

# Traverse each day folder
for day_folder in os.listdir(BASE_DIR):
    day_path = os.path.join(BASE_DIR, day_folder)
    if not os.path.isdir(day_path):
        continue

    for filename in os.listdir(day_path):
        match = pattern.match(filename)
        if match:
            imo = match.group(1)
            imo_set.add(imo)

# Print or use the unique IMO set
print(f"Found {len(imo_set)} unique IMOs.")

Found 1149 unique IMOs.


In [70]:
# -------- Configuration --------
BASE_DIR = "./data/trips"
IMO = "6617855"  # example IMO, update this
FILENAME = f"IMO_{IMO}.csv"

# -------- Load all matching CSVs across days --------
all_dfs = []

for day_folder in sorted(os.listdir(BASE_DIR)):
    day_path = os.path.join(BASE_DIR, day_folder)
    if not os.path.isdir(day_path):
        continue

    imo_path = os.path.join(day_path, FILENAME)
    if os.path.exists(imo_path):
        df = pd.read_csv(imo_path)
        df["source_day"] = day_folder  # optional: keep track of origin
        all_dfs.append(df)

if not all_dfs:
    raise FileNotFoundError(f"No data found for IMO {IMO}")

full_df = pd.concat(all_dfs, ignore_index=True)

# -------- Preprocessing --------
# Ensure timestamps are in datetime format
full_df["# Timestamp"] = pd.to_datetime(full_df["# Timestamp"])  # Adjust column name if needed
full_df = full_df.sort_values("# Timestamp").reset_index(drop=True)

# -------- Voyage Segmentation --------
# Define threshold for gap between pings (e.g., 4 hours) to indicate new voyage
GAP_HOURS = 4
GAP = timedelta(hours=GAP_HOURS)

# Compute time differences between consecutive points
full_df["TimeDiff"] = full_df["# Timestamp"].diff()
full_df["NewVoyage"] = (full_df["TimeDiff"] > GAP) | (full_df["TimeDiff"].isna())

# Assign voyage ID
full_df["VoyageID"] = full_df["NewVoyage"].cumsum().astype(int)

# -------- Result --------
# Show summary of voyages
voyage_summary = full_df.groupby("VoyageID")["# Timestamp"].agg(["min", "max", "count"])
display(voyage_summary)

# -------- Config --------
PROCESSED_DIR = "../notebooks/data/processed_trips"
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Make sure 'IMO' is a scalar (from the grouped data or filename)
imo_number = full_df["IMO"].iloc[0]  # assuming consistent IMO per track

# -------- Save Each Voyage --------
for i, voyage_id in enumerate(sorted(full_df["VoyageID"].unique()), start=1):
    voyage_df = full_df[full_df["VoyageID"] == voyage_id]
    out_filename = f"IMO_{imo_number}_trip_{i}.csv"
    out_path = os.path.join(PROCESSED_DIR, out_filename)
    voyage_df.to_csv(out_path, index=False)

print(f"Saved {i} voyages for IMO {imo_number} to {PROCESSED_DIR}")

,min,max,count
VoyageID,,,
1,2025-01-01 00:00:10,2025-01-01 23:59:55,12485
2,2025-02-01 00:00:24,2025-02-01 02:07:38,1512
3,2025-04-01 16:39:08,2025-04-01 23:59:40,1496
